In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

### 因子构造样例

#### 预先读取merge_daily_info

In [2]:
merge_daily_info = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/merge_daily_info.parquet')
merge_daily_info['TradingDate'] = pd.to_datetime(merge_daily_info['TradingDate'])
merge_daily_info = merge_daily_info.drop_duplicates(subset=['Stkcd','TradingDate'])
TRD_Dalyr = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/TRD_Dalyr.parquet')

In [3]:
TRD_Dalyr.columns

Index(['Stkcd', 'TradingDate', 'Opnprc', 'Hiprc', 'Loprc', 'Clsprc',
       'Dnshrtrd', 'Dnvaltrd', 'Dsmvosd', 'Dsmvtll', 'Dretwd', 'Dretnd',
       'Adjprcwd', 'Adjprcnd', 'Markettype', 'Capchgdt', 'Trdsta',
       'Ahshrtrd_D', 'Ahvaltrd_D', 'PreClosePrice', 'ChangeRatio', 'LimitDown',
       'LimitUp', 'LimitStatus'],
      dtype='object')

In [4]:
#数据提取与合并
Hiprc = TRD_Dalyr[['TradingDate','Stkcd','Hiprc']]
Dnshrtrd = TRD_Dalyr[['TradingDate','Stkcd','Dnshrtrd']]
Clsprc = TRD_Dalyr[['TradingDate','Stkcd','Clsprc']]
calc_cvturn = pd.merge(Hiprc,Dnshrtrd ,on=['TradingDate','Stkcd'],how='left')
calc_cvturn = pd.merge(calc_cvturn,Clsprc ,on=['TradingDate','Stkcd'],how='left')
# 现在 calc_factor_data 应该只包含 'TradingDate', 'Stkcd', 'Opnprc', 'Clsprc'
calc_cvturn.columns

Index(['TradingDate', 'Stkcd', 'Hiprc', 'Dnshrtrd', 'Clsprc'], dtype='object')

# alpha022

In [5]:
COLUMN_FACTOR_NAME = "alpha022" 
ALPHA_SAVE_PATH_DIR = "/Users/xuyanye/Desktop/quant mm/factor/alpha"
if not os.path.exists(ALPHA_SAVE_PATH_DIR):
    os.makedirs(ALPHA_SAVE_PATH_DIR)
# 定义最终因子在DataFrame中的列名，以及Parquet文件的基础名称
# 这两部分必须一致，才能被您的回测代码正确识别

In [6]:
calc_factor_data = calc_cvturn.copy()

# 1. 确保数据按股票代码和交易日期排序
# 这一步对于计算时间序列相关的函数（如 diff, rolling, shift）至关重要
calc_factor_data = calc_factor_data.sort_values(by=['Stkcd', 'TradingDate']).copy()

# 2. 计算滚动5日高价和成交股数的关联系数 (correlation)
# min_periods=1 允许在数据不足5期时也进行计算
calc_factor_data['corr_high_volume_5'] = calc_factor_data.groupby('Stkcd').apply(
    lambda x: x['Hiprc'].rolling(window=5, min_periods=1).corr(x['Dnshrtrd'])
).reset_index(level=0, drop=True)

# 3. 计算5日关联系数的5期差分 (delta)
# 这将计算当前5日关联系数与5期前5日关联系数的变化量。如果此值降低（负值），则表明相关性减弱或反向程度增加。
calc_factor_data['delta_corr_high_volume_5_5'] = calc_factor_data.groupby('Stkcd')['corr_high_volume_5'].diff(5)

# 4. 计算滚动20日收盘价的标准差 (stddev)
# min_periods=1 允许在数据不足20期时也进行计算。标准差越大，表示波动性越大。
calc_factor_data['stddev_close_20'] = calc_factor_data.groupby('Stkcd')['Clsprc'].rolling(window=20, min_periods=1).std().reset_index(level=0, drop=True)

# 5. 对20日收盘价标准差进行跨截面（每天）排名 (rank)
# 'method="average"' 是 pandas 默认的排名方式，'ascending=True' 默认是升序排名。排名越大，表示波动性在所有股票中相对越大。
calc_factor_data['rank_stddev_close_20'] = calc_factor_data.groupby('TradingDate')['stddev_close_20'].rank(method='average', ascending=True)

# 6. 对排名结果进行标准化 (standardization)，这里使用Z-score标准化
# 按照每天进行分组，对每个股票的排名进行标准化。标准化确保排名值具有可比性。
calc_factor_data['std_rank_stddev_close_20'] = calc_factor_data.groupby('TradingDate')['rank_stddev_close_20'].transform(lambda x: (x - x.mean()) / x.std())

# 7. 计算因子中间项：delta(correlation(high, volume, 5), 5) * rank(stddev(close,20))
# 我们的目标是：
#   - 当 delta_corr_high_volume_5_5 降低（负值，表示量价反向程度增加）时，因子倾向于投资。
#   - 当 std_rank_stddev_close_20 增加（正值，表示波动率增加）时，因子倾向于投资。
# 因此，当 delta_corr_high_volume_5_5 为负且 std_rank_stddev_close_20 为正时，它们的乘积 product_term 将为负。
calc_factor_data['product_term'] = calc_factor_data['delta_corr_high_volume_5_5'] * calc_factor_data['std_rank_stddev_close_20']

# 8. 计算最终的因子值：-1 * product_term
# 由于我们希望当 product_term 为负时（即量价反向程度增加且波动率增加时）因子值变大，所以乘以 -1。
# 这样，因子值越大，代表越倾向于投资。
calc_factor_data['alpha_new_factor'] = -1 * calc_factor_data['product_term']

/var/folders/6j/sy28sk9165s6btqff5dbsdmc0000gn/T/ipykernel_14358/2217402216.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  calc_factor_data['corr_high_volume_5'] = calc_factor_data.groupby('Stkcd').apply(


In [8]:
# 假设 merge_daily_info 是另一个包含 Stkcd 和 TradingDate 的 DataFrame
# 将计算出的 Alpha08_Factor 合并到 merge_daily_info
# 注意：如果 calc_cvturn 就是最终要用的 DataFrame，可能不需要这一步合并
# 这里为了与用户提供的格式保持一致，假设有一个目标 DataFrame 'merge_daily_info'
# 如果 calc_cvturn 本身就是最终结果，则可以跳过此合并步骤
# 请根据实际情况调整 'merge_daily_info' 的定义
try:
    # 模拟 merge_daily_info 如果它不存在
    if 'merge_daily_info' not in locals():
        print("警告：'merge_daily_info' 未定义。为演示目的，将假设 merge_daily_info 等同于 calc_cvturn 的基础部分。")
        merge_daily_info = calc_cvturn[['Stkcd', 'TradingDate']].copy() # 只是一个示例

    # 进行合并
    calc_cvturn = pd.merge(merge_daily_info,
                           calc_factor_data[['Stkcd','TradingDate','alpha_new_factor']],
                           on=['Stkcd','TradingDate'],
                           how='left')
    calc_cvturn = calc_cvturn.rename(columns={'alpha_new_factor': 'alpha022'})
except NameError:
    print("错误：'merge_daily_info' 未定义，请确保该DataFrame存在后再执行合并。")


# 去掉缺失值与inf (根据用户提供的格式)
# 这一步会移除 Alpha12_Factor 列中因为 delta 计算导致的 NaN 值 (例如每个股票的第一个交易日)
calc_cvturn = calc_cvturn.dropna(subset=['alpha022']) # 只针对因子列去除NaN，避免影响其他数据
calc_cvturn = calc_cvturn[~np.isinf(calc_cvturn['alpha022'])] # 确保因子值不是无穷大

# 打印结果 DataFrame 的头部，以供检查
print(calc_cvturn.head())

    Stkcd TradingDate  alpha022
6       1  1991-04-10  0.491139
7       1  1991-04-12  0.031699
9       1  1991-04-16  0.329332
10      1  1991-04-17  0.411163
11      1  1991-04-18  1.315259


In [10]:
##采用parquet优化内存
save_path = r'/Users/xuyanye/Desktop/quant mm/factor/alpha/alpha022.parquet'
calc_cvturn.to_parquet(save_path, compression='gzip')